In [2]:
!pip install -Uqq sagemaker transformers[torch] pytorch-lightning datasets wandb


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
distributed 2022.7.0 requires tornado<6.2,>=6.0.3, but you have tornado 6.3.3 which is incompatible.


In [3]:
import sagemaker

sagemaker_session = sagemaker.Session()
region = sagemaker_session.boto_region_name

bucket = sagemaker_session.default_bucket()
prefix = "sagemaker/food101"

role = sagemaker.get_execution_role()

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [5]:
!pip install torchvision

  Obtaining dependency information for torchvision from https://files.pythonhosted.org/packages/84/eb/4f6483ae9094e164dc5b9b792e377f7d37823b0bedc3eef3193d416d2bb6/torchvision-0.16.0-cp310-cp310-manylinux1_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 33.7 MB/s eta 0:00:0000:01:00:01


In [ ]:
from getpass import getpass

wandb_logging = True

if wandb_logging:
    wandb_api_key = getpass("Copy your WANDB_API_KEY: ")

In [ ]:
import wandb
from pytorch_lightning.loggers import WandbLogger

if wandb_logging:
    wandb.login(key=wandb_api_key, relogin=True)

    logger = WandbLogger(
        project="Foodformer",
        name="VisionTransformer-base",
        checkpoint_name="vit",
        log_model=True,
        save_dir=".",
    )
else:
    logger = None

In [ ]:
from functools import partial

from torchvision.datasets import Food101
from transformers import ViTFeatureExtractor

model_name_or_path = 'google/vit-base-patch16-224-in21k'
feature_extractor = ViTFeatureExtractor.from_pretrained(model_name_or_path)

preprocessor = partial(feature_extractor, return_tensors='pt')

train_ds = Food101(root="food101_dataset", split='train', transform=preprocessor, download=True)
test_ds = Food101(root="food101_dataset", split='test', transform=preprocessor)

labels = train_ds.classes

/opt/conda/lib/python3.10/site-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


100%|██████████| 4996278331/4996278331 [04:38<00:00, 17948913.66it/s]


Extracting food101_dataset/food-101.tar.gz to food101_dataset


In [ ]:
names = ["train", "test"]
datasets = [train_ds, test_ds]

# 🏺 create our Artifact
raw_data = wandb.Artifact(
    "food101-train",
    type="dataset",
    description="Custom Food101 dataset, split into train/test",
    metadata={"sizes": [len(dataset) for dataset in datasets]},
)

for name, data in zip(names, datasets):
    with raw_data.new_file(name + ".pt", mode="wb") as fs:
        torch.save(data, fs)

# ✍️ Save the artifact to W&B.
wandb.run.log_artifact(raw_data)

In [ ]:
inputs = sagemaker_session.upload_data(path="./food101_dataset", bucket=bucket, key_prefix=prefix)
print("input spec (in this case, just an S3 path): {}".format(inputs))

In [ ]:
import sagemaker
from sagemaker.pytorch import PyTorch

sagemaker_session = sagemaker.Session()

pytorch_estimator = PyTorch(
    entry_point='train.py',
    instance_type='ml.g4dn.2xlarge',
    instance_count=1,
    framework_version='2.0.0',
    py_version='py310',
    output_path="s3://mlops-course-wei",
    role=role,
    dependencies=['requirements.txt'],
    # source_dir=".",
    # hyperparameters={},
)

In [ ]:
pytorch_estimator.fit({"train": f"s3://{bucket}/{prefix}"}, wait=True)

In [ ]:
predictor = pytorch_estimator.deploy(initial_instance_count=1, instance_type="ml.g4dn.2xlarge")

In [ ]:
sagemaker_session.delete_endpoint(endpoint_name=predictor.endpoint_name)

In [ ]:
wandb.finish()